## Playing around with LangChain on Knowledge Graphs, specifically on a Neo4j database

-------

Credits:
* https://python.langchain.com/docs/use_cases/graph/quickstart/ 

In [1]:
import getpass
import os

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

········


### Setting up and connecting to the Neo4j database instance

In [17]:
#details from sandbox setup to establish the connection with the Neo4j database.

os.environ["NEO4J_URI"] = "bolt://3.237.86.115:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "acceptance-radians-fines"

In [18]:
from langchain_community.graphs import Neo4jGraph

In [19]:
graph = Neo4jGraph()

In [13]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://3.237.86.115:7687",
  auth=basic_auth("neo4j", "acceptance-radians-fines"))

cypher_query = '''
MATCH (n)
RETURN COUNT(n) AS count
LIMIT $limit
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      limit=10).data())
  for record in results:
    print(record['count'])

driver.close()

<ipython-input-13-a53ff26dcca8>:14: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


0


In [20]:
#import movie information from Neo4j example database.

movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

graph.query(movies_query)

[]

In [21]:
#instantiating the graph schema so that I can run queries on it.

graph.refresh_schema()
#command to refresh the schema information if changes are made to the graph later.

print(graph.schema)

Node properties are the following:
Movie {imdbRating: FLOAT, id: STRING, released: DATE, title: STRING},Person {name: STRING},Genre {name: STRING}
Relationship properties are the following:

The relationships are the following:
(:Movie)-[:IN_GENRE]->(:Genre),(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)


### Implementing the built-in LangChain chain (OpenAI API) specifically for Cypher queries

In [22]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

User inputted question triggers the generation of a Cypher query statement that queries the Neo4j movie database that has been set up.

The result from the database query is converted to natural language, producing the displayed answer.

In [24]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "What was the cast of the Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Movie {title: "Casino"})<-[:ACTED_IN]-(actor:Person)
RETURN actor.name
Full Context:
[{'actor.name': 'Sharon Stone'}, {'actor.name': 'Joe Pesci'}, {'actor.name': 'James Woods'}, {'actor.name': 'Robert De Niro'}]

> Finished chain.


{'query': 'What was the cast of the Casino?',
 'result': 'The cast of Casino included Sharon Stone, Joe Pesci, James Woods, and Robert De Niro.'}

In [25]:
#Validating the answer (relationship direction)

chain = GraphCypherQAChain.from_llm(
    graph=graph, llm=llm, verbose=True, validate_cypher=True
)
response = chain.invoke({"query": "What was the cast of the Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Movie {title: "Casino"})<-[:ACTED_IN]-(actor:Person)
RETURN actor.name
Full Context:
[{'actor.name': 'Sharon Stone'}, {'actor.name': 'Joe Pesci'}, {'actor.name': 'James Woods'}, {'actor.name': 'Robert De Niro'}]

> Finished chain.


{'query': 'What was the cast of the Casino?',
 'result': 'The cast of Casino included Sharon Stone, Joe Pesci, James Woods, and Robert De Niro.'}